# Clustering Crypto

In [5]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [9]:
# Load the crypto_data.csv dataset.
file_path = "C:/Users/pooja/Desktop/RESOURCES/crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.info()
crypto_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252 entries, 0 to 1251
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1252 non-null   object 
 1   CoinName         1252 non-null   object 
 2   Algorithm        1252 non-null   object 
 3   IsTrading        1252 non-null   bool   
 4   ProofType        1252 non-null   object 
 5   TotalCoinsMined  744 non-null    float64
 6   TotalCoinSupply  1252 non-null   object 
dtypes: bool(1), float64(1), object(5)
memory usage: 60.0+ KB


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [10]:
# Keep all the cryptocurrencies that are being traded.
crypto_df1 = crypto_df[crypto_df["IsTrading"] == True]
crypto_df1.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [12]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df['Algorithm'].isnull().values.any()


False

In [13]:
# Remove the "IsTrading" column. 
crypto_df.drop(['IsTrading'], axis=1, inplace=True)
crypto_df.info()
crypto_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252 entries, 0 to 1251
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1252 non-null   object 
 1   CoinName         1252 non-null   object 
 2   Algorithm        1252 non-null   object 
 3   ProofType        1252 non-null   object 
 4   TotalCoinsMined  744 non-null    float64
 5   TotalCoinSupply  1252 non-null   object 
dtypes: float64(1), object(5)
memory usage: 58.8+ KB


,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [14]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna(how='any')
crypto_df.shape

(744, 6)

In [15]:
# Keep the rows where coins are mined.
crypto_df.drop(crypto_df[crypto_df.TotalCoinsMined <= 0].index, inplace=True)
crypto_df.shape

(577, 6)

In [16]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coins_name = pd.DataFrame(crypto_df["CoinName"], index=crypto_df.index)
coins_name.head(5)

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [17]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop("CoinName", axis=1, inplace=True)
crypto_df.head(5)

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethash,PoW,1.076842e+08,0


In [18]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df)
X.head()

,TotalCoinsMined,Unnamed: 0_1337,Unnamed: 0_1CR,Unnamed: 0_404,Unnamed: 0_42,Unnamed: 0_8BIT,Unnamed: 0_AAC,Unnamed: 0_ABJ,Unnamed: 0_ABS,Unnamed: 0_ABY,...,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
0,4.199995e+01,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.055185e+09,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2.927942e+10,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1.792718e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:20])

[[-0.11450078 -0.04166667 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [-0.09041991 -0.04166667 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [ 0.55369883 24.         -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 ...
 [-0.11422223 -0.04166667 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [-0.11408691 -0.04166667 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [-0.09999947 -0.04166667 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [21]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

In [25]:
# Create a DataFrame with the three principal components.
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.42558153, -0.01183048, -0.47821475],
       [-0.49773125, -0.00950399, -0.52144663],
       [ 0.56648838, -0.04832998, -0.22062923],
       ...,
       [-0.3840716 , -0.17555845,  0.44149733],
       [-0.28103485, -0.01686931, -0.08598968],
       [-0.30313893, -0.04909421, -0.28075356]])

In [27]:
# Transform PCA data to a DataFrame named “pcs_df” that includes the following columns: PC 1, PC 2, and PC 3. Use the crypto_df.index as the index for this new DataFrame
pcs_df=pd.DataFrame(data=X_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
pcs_df.head()

,PC 1,PC 2,PC 3
0,-0.425582,-0.011830,-0.478215
2,-0.497731,-0.009504,-0.521447
5,0.566488,-0.048330,-0.220629
7,-0.142007,-0.200097,0.159880
8,-0.039513,-0.329581,0.273136


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [28]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_data)
elbow_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


C:\Users\pooja\anaconda3\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [35]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

In [36]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.merge(crypto_df, pcs_df, left_index=True, right_index=True)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.merge(clustered_df, coins_name, left_index=True, right_index=True)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# Add the predicted class columns
pcs_df["class"] = model.labels_
pcs_df.head()

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(577, 10)


,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName
0,42,Scrypt,PoW/PoS,4.199995e+01,42,-0.425582,-0.011830,-0.478215,0,42 Coin
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.497731,-0.009504,-0.521447,0,404Coin
5,1337,X13,PoW/PoS,2.927942e+10,314159265359,0.566488,-0.048330,-0.220629,0,EliteCoin
7,BTC,SHA-256,PoW,1.792718e+07,21000000,-0.142007,-0.200097,0.159880,0,Bitcoin
8,ETH,Ethash,PoW,1.076842e+08,0,-0.039513,-0.329581,0.273136,0,Ethereum
9,LTC,Scrypt,PoW,6.303924e+07,84000000,-0.484642,-0.070813,-0.005082,0,Litecoin
10,DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.114225,-0.266022,-0.475223,0,Dash
11,XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.209823,-0.121747,0.435742,0,Monero
12,ETC,Ethash,PoW,1.133597e+08,210000000,-0.246760,-0.327555,0.332769,0,Ethereum Classic
13,ZEC,Equihash,PoW,7.383056e+06,21000000,-0.384268,-0.175112,0.441490,0,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [38]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    z="ProofType",
    color="class",
    symbol="class",   
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1)) 
fig.show()



In [39]:
# Create a table with tradable cryptocurrencies.
clustered_df1= clustered_df[['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class']]
clustered_df1.hvplot.table(width=800)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [19]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

There are 532 tradable cryptocurrencies.


In [43]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:20])

[[-0.11450078 -0.04166667 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [-0.09041991 -0.04166667 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [ 0.55369883 24.         -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 ...
 [-0.11422223 -0.04166667 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [-0.11408691 -0.04166667 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [-0.09999947 -0.04166667 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]]


In [52]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df=pd.DataFrame(clustered_df, columns=["Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinMined"], index=crypto_df.index)
plot_df.head()

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df=pd.DataFrame(clustered_df, columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinMined"], index=crypto_df.index)
plot_df.head()

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df=pd.DataFrame(clustered_df, columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinMined", "class"], index=crypto_df.index)
plot_df.head()

plot_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinMined,class
0,42 Coin,Scrypt,PoW/PoS,42,NaN,0
2,404Coin,Scrypt,PoW/PoS,532000000,NaN,0
5,EliteCoin,X13,PoW/PoS,314159265359,NaN,0
7,Bitcoin,SHA-256,PoW,21000000,NaN,0
8,Ethereum,Ethash,PoW,0,NaN,0
9,Litecoin,Scrypt,PoW,84000000,NaN,0
10,Dash,X11,PoW/PoS,22000000,NaN,0
11,Monero,CryptoNight-V7,PoW,0,NaN,0
12,Ethereum Classic,Ethash,PoW,210000000,NaN,0
13,ZCash,Equihash,PoW,21000000,NaN,0


In [40]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
clustered_df.hvplot.scatter(
x="TotalCoinsMined",
y="TotalCoinSupply",
hover_cols=["CoinName"],
by="class",
)


:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)